In [ ]:
import numpy as np
from pathlib import Path
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
from aicsimageio.writers import OmeTiffWriter

import sys
src_path = str(Path.cwd().parent)
if src_path not in sys.path:
    sys.path.append(src_path)
from src.d00_utils.utilities import construct_ome_metadata

In [ ]:
multiexp_dirpath = Path(input())

In [ ]:
ch_subset = [0, 1, 2, 5]

In [ ]:
def create_subsets(exp_dirpath, ch_subset):
    exp_dirpath = Path(exp_dirpath)
    imgpaths = [path for path in exp_dirpath.glob('*.ome.tif')]
    imgsubset_dirpath = exp_dirpath.parent / 'raw_ometifs_subset'
    imgsubset_dirpath.mkdir(parents=True, exist_ok=True)

    for imgpath in imgpaths:
        img_file = AICSImage(imgpath, reader=OmeTiffReader)
        img = img_file.data
        for ch in ch_subset:
            img_chs = [img[:, ch, np.newaxis, :, :, :] for ch in ch_subset]
            imgsubset = np.concatenate(img_chs, axis=1)

        ome_metadata = construct_ome_metadata(imgsubset, img_file.physical_pixel_sizes, channel_names=None)
        OmeTiffWriter.save(imgsubset, imgsubset_dirpath / imgpath.name,
                                   ome_xml=ome_metadata)
    print('Done!')

In [ ]:
exp_dirpaths = [(dirpath / 'raw_ometifs') for dirpath in multiexp_dirpath.iterdir() if dirpath.is_dir()]
num_dirpaths = len(exp_dirpaths)
for i, exp_dirpath in enumerate(exp_dirpaths):
    print(f'Creating subsets for file {i+1}/{num_dirpaths}')
    create_subsets(exp_dirpath, ch_subset)
print('Finished with all files')
    